In [ ]:

#use for image processing allow reding writing and manipulating image task like resizing colour conversion image filtering edge detection
#linear stack of layers can create neural networks by adding layers to this model one by one suitable for cnn architectures
#creates a convolutional kernal that is convolved with input layer to produce an output tensor core cnn building block responsible for feature extraction
#reduce spatial dimension of the input tensor help to reduce no of parameter and computation in the network 
#a fully connected layer which connects every neuron in one layer to other in next neuron used in final stage for classification tasks
#regularization technique randomly selected neurons are ignored during training avoid overfitting
#flattens the input convert multidimensinal tensor to one dimentional vector ehich is then fed into fully connected layer
#adaptive moment estimation . it combines the advantages of (adagrad&rmspromp) computes individual adaptive learing rate for 
##diff parameters making effective for deep neural network
#provide real time data augmentation and preprocessing ,generate batch of images with real time data augmentation which improves 
# #the robustness and generalize model capability allow operations like rotation ,shift ,flip
# import required packages
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

# Initialize image data generator with rescaling
# creates an instance ,rescale the parameter pixel value of image ,each pixel value will be multiplied by 1/255 this will normalize the range b/w 0-1
# preprocessing step in dl , ensures input value lie between a consistent range 
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all train images
#supervised model with labelled data , each image belongs to a class,labels are one hot encoded vectors suitable for multiclass classification tasks
## labels look li [1,0,0],[0,1,0],[0,0,1]
train_generator = train_data_gen.flow_from_directory(
        'data/train',   # Directory that contains all the trained images 
        target_size=(48, 48), #ensures all image have same dimensions required for batch processing and feeding in neural networks
        batch_size=64, #no of image to be yield in each batch in each iteration generator provides 64 images at a time
        color_mode="grayscale",#loads greyscale image single channel colour useful where colour not necessary
        class_mode='categorical')

# Preprocess all test images
#yield batch of data model use data to compute performance metrics like accuracy&loss on unseen data , helps to calculate generalization capability
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',  #Directory that contains all the test images
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# create model structure
#initialize new instance of the sequential class from keras creating an empty model this will be used to build neural network by adding layer in sequential odr
emotion_model = Sequential()
 
# First CNN Layer
#rectified linear unt helps to learn complex patterns ,32filter,kernel size3x3,input shape height and width of image
emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))#64 filters
emotion_model.add(MaxPooling2D(pool_size=(2, 2))) # Picks up the maximum value from the pool and forms output
emotion_model.add(Dropout(0.25)) # To avoid overfitting 25% neurons

# Second CNN Layer
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu')) 
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25)) # To avoid overfitting
#add a dense with 1024 neurons
#add dense with 7neurons(emotions) ,softmax outputs the probability distribution over 7 classes for multifunction
##classification calculate max probability and emotion with max probability is displayed
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu')) # Fully connected neural layer
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy']) # Compiling the model
#yields batch of training data and corresponding labels indefinetly in a loop
# Train the neural network/model
emotion_model_info = emotion_model.fit_generator(
        train_generator,
        steps_per_epoch=28709 // 64,#specify no of batch samples per epoch tarining sample = 28709 , 64 batch size ,448 steps in each epoch 
        epochs=8,#no of times to iterate over training data 
        validation_data=validation_generator,#evaluate performance after each epoch 
        validation_steps=7178 // 64)#no of validation samples to use per epoch 7178 no of validation samples 64 batch size 112 no of validation steps 

# save model structure in jason file
model_json = emotion_model.to_json()
with open('model/emotion_model.json', "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('model/emotion_model.h5')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


C:\Users\deepa\anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\deepa\AppData\Local\Temp\ipykernel_9972\2545805594.py:55: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  emotion_model_info = emotion_model.fit_generator(


Epoch 1/8
448/448 [==============================] - 434s 942ms/step - loss: 1.8100 - accuracy: 0.2522 - val_loss: 1.7406 - val_accuracy: 0.2870
Epoch 2/8
448/448 [==============================] - 410s 915ms/step - loss: 1.6508 - accuracy: 0.3491 - val_loss: 1.5607 - val_accuracy: 0.4042
Epoch 3/8
448/448 [==============================] - 26s 57ms/step - loss: 1.5472 - accuracy: 0.4071 - val_loss: 1.4765 - val_accuracy: 0.4424
Epoch 4/8
448/448 [==============================] - 20s 45ms/step - loss: 1.4742 - accuracy: 0.4331 - val_loss: 1.4166 - val_accuracy: 0.4562
Epoch 5/8
448/448 [==============================] - 21s 47ms/step - loss: 1.4186 - accuracy: 0.4578 - val_loss: 1.3729 - val_accuracy: 0.4849
Epoch 6/8
448/448 [==============================] - 22s 48ms/step - loss: 1.3651 - accuracy: 0.4795 - val_loss: 1.3321 - val_accuracy: 0.4954
Epoch 7/8
240/448 [===============>..............] - ETA: 8s - loss: 1.3234 - accuracy: 0.4971